In [26]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

In [31]:
df = pd.read_csv('data/ckd.csv', index_col=0)
df.columns = ['age', 'blood_pressure', 'specific_gravity', 'albumin', 'sugar', 'abnormal_red_blood_cells',
              'abnormal_pus_cell', 'pus_cell_clumps', 'bacteria', 'blood_glucose_random', 'blood_urea',
              'serum_creatinine', 'sodium', 'potassium', 'haemoglobin', 'packed_cell_volume', 
              'white_blood_cell_count', 'red_blood_cell_count', 'hypertension', 'diabetes_mellitus', 
              'coronary_artery_disease', 'poor_appetite', 'peda_edema', 'aanemia', 'class']
df.head()

,age,blood_pressure,specific_gravity,albumin,sugar,abnormal_red_blood_cells,abnormal_pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,blood_urea,serum_creatinine,sodium,potassium,haemoglobin,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,poor_appetite,peda_edema,aanemia,class
id,,,,,,,,,,,,,,,,,,,,,,,,,
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,36.0,1.2,NaN,NaN,15.4,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,18.0,0.8,NaN,NaN,11.3,38,6000,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,53.0,1.8,NaN,NaN,9.6,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,56.0,3.8,111.0,2.5,11.2,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,26.0,1.4,NaN,NaN,11.6,35,7300,4.6,no,no,no,good,no,no,ckd


In [32]:
df['packed_cell_volume'] = pd.to_numeric(df['packed_cell_volume'], errors='coerce')
df['white_blood_cell_count'] = pd.to_numeric(df['white_blood_cell_count'], errors='coerce')
df['red_blood_cell_count'] = pd.to_numeric(df['red_blood_cell_count'], errors='coerce')
df['diabetes_mellitus'].replace(to_replace = {'\tno':'no','\tyes':'yes',' yes':'yes'},inplace=True)
df['coronary_artery_disease'] = df['coronary_artery_disease'].replace(to_replace = '\tno', value='no')
df['class'] = df['class'].replace(to_replace = {'ckd\t': 'ckd', 'notckd': 'not ckd'})

categorical_columns = ['abnormal_red_blood_cells', 'abnormal_pus_cell', 'pus_cell_clumps', 
                       'bacteria', 'hypertension', 'diabetes_mellitus', 'coronary_artery_disease', 
                       'poor_appetite', 'peda_edema', 'aanemia', 'class']

replace_dict = {'normal': 0, 'abnormal': 1, 'notpresent': 0, 'present': 1, 'no': 0, 'yes': 1,
               'good': 0, 'poor': 1, 'not ckd': 0, 'ckd': 1}

for column in categorical_columns:
    df[column] = df[column].replace(replace_dict)

In [35]:
df_knn = pd.get_dummies(df.drop(columns=['class']), dummy_na=True)
scaler = MinMaxScaler(feature_range=(0, 1))
df_knn = pd.DataFrame(scaler.fit_transform(df_knn), columns = df_knn.columns)
knn_imputer = KNNImputer(n_neighbors=len(df_knn.columns))
df_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(df_knn), columns=df_knn.columns)
df_knn_imputed = scaler.inverse_transform(df_knn_imputed)
df_knn_imputed = pd.DataFrame(df_knn_imputed, columns=df_knn.columns)
df_knn_imputed['class'] = df['class']
df_knn_imputed[categorical_columns] = df_knn_imputed[categorical_columns].round(0)
df_knn_imputed

,age,blood_pressure,specific_gravity,albumin,sugar,abnormal_red_blood_cells,abnormal_pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,blood_urea,serum_creatinine,sodium,potassium,haemoglobin,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,poor_appetite,peda_edema,aanemia,class
0,48.000000,80.000000,1.020000,1.000000,0.000000,0.0,0.0,0.0,0.0,121.000000,36.000000,1.200000,137.708333,4.391667,15.400000,44.000000,7800.000000,5.200000,1.0,1.0,0.0,0.0,0.0,0.0,1
1,7.000000,50.000000,1.020000,4.000000,0.000000,0.0,0.0,0.0,0.0,110.958333,18.000000,0.800000,140.500000,4.162500,11.300000,38.000000,6000.000000,5.329167,0.0,0.0,0.0,0.0,0.0,0.0,1
2,62.000000,80.000000,1.010000,2.000000,3.000000,0.0,0.0,0.0,0.0,423.000000,53.000000,1.800000,134.291667,4.512500,9.600000,31.000000,7500.000000,3.987500,0.0,1.0,0.0,1.0,0.0,1.0,1
3,48.000000,70.000000,1.005000,4.000000,0.000000,0.0,1.0,1.0,0.0,117.000000,56.000000,3.800000,111.000000,2.500000,11.200000,32.000000,6700.000000,3.900000,1.0,0.0,0.0,1.0,1.0,1.0,1
4,51.000000,80.000000,1.010000,2.000000,0.000000,0.0,0.0,0.0,0.0,106.000000,26.000000,1.400000,140.000000,4.175000,11.600000,35.000000,7300.000000,4.600000,0.0,0.0,0.0,0.0,0.0,0.0,1
5,60.000000,90.000000,1.015000,3.000000,0.000000,0.0,0.0,0.0,0.0,74.000000,25.000000,1.100000,142.000000,3.200000,12.200000,39.000000,7800.000000,4.400000,1.0,1.0,0.0,0.0,1.0,0.0,1
6,68.000000,70.000000,1.010000,0.000000,0.000000,0.0,0.0,0.0,0.0,100.000000,54.000000,24.000000,104.000000,4.000000,12.400000,36.000000,8183.333333,5.179167,0.0,0.0,0.0,0.0,0.0,0.0,1
7,24.000000,82.500000,1.015000,2.000000,4.000000,0.0,1.0,0.0,0.0,410.000000,31.000000,1.100000,136.958333,5.550000,12.400000,44.000000,6900.000000,5.000000,0.0,1.0,0.0,0.0,1.0,0.0,1
8,52.000000,100.000000,1.015000,3.000000,0.000000,0.0,1.0,1.0,0.0,138.000000,60.000000,1.900000,133.291667,6.508333,10.800000,33.000000,9600.000000,4.000000,1.0,1.0,0.0,0.0,0.0,1.0,1
9,53.000000,90.000000,1.020000,2.000000,0.000000,1.0,1.0,1.0,0.0,70.000000,107.000000,7.200000,114.000000,3.700000,9.500000,29.000000,12100.000000,3.700000,1.0,1.0,0.0,1.0,0.0,1.0,1


In [36]:
df_knn_imputed.to_csv('data/ckd_clean.csv', index=False)

In [37]:
df_clean = pd.read_csv('data/ckd_clean.csv')
df_clean

,age,blood_pressure,specific_gravity,albumin,sugar,abnormal_red_blood_cells,abnormal_pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,blood_urea,serum_creatinine,sodium,potassium,haemoglobin,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,poor_appetite,peda_edema,aanemia,class
0,48.000000,80.000000,1.020000,1.000000,0.000000,0.0,0.0,0.0,0.0,121.000000,36.000000,1.200000,137.708333,4.391667,15.400000,44.000000,7800.000000,5.200000,1.0,1.0,0.0,0.0,0.0,0.0,1
1,7.000000,50.000000,1.020000,4.000000,0.000000,0.0,0.0,0.0,0.0,110.958333,18.000000,0.800000,140.500000,4.162500,11.300000,38.000000,6000.000000,5.329167,0.0,0.0,0.0,0.0,0.0,0.0,1
2,62.000000,80.000000,1.010000,2.000000,3.000000,0.0,0.0,0.0,0.0,423.000000,53.000000,1.800000,134.291667,4.512500,9.600000,31.000000,7500.000000,3.987500,0.0,1.0,0.0,1.0,0.0,1.0,1
3,48.000000,70.000000,1.005000,4.000000,0.000000,0.0,1.0,1.0,0.0,117.000000,56.000000,3.800000,111.000000,2.500000,11.200000,32.000000,6700.000000,3.900000,1.0,0.0,0.0,1.0,1.0,1.0,1
4,51.000000,80.000000,1.010000,2.000000,0.000000,0.0,0.0,0.0,0.0,106.000000,26.000000,1.400000,140.000000,4.175000,11.600000,35.000000,7300.000000,4.600000,0.0,0.0,0.0,0.0,0.0,0.0,1
5,60.000000,90.000000,1.015000,3.000000,0.000000,0.0,0.0,0.0,0.0,74.000000,25.000000,1.100000,142.000000,3.200000,12.200000,39.000000,7800.000000,4.400000,1.0,1.0,0.0,0.0,1.0,0.0,1
6,68.000000,70.000000,1.010000,0.000000,0.000000,0.0,0.0,0.0,0.0,100.000000,54.000000,24.000000,104.000000,4.000000,12.400000,36.000000,8183.333333,5.179167,0.0,0.0,0.0,0.0,0.0,0.0,1
7,24.000000,82.500000,1.015000,2.000000,4.000000,0.0,1.0,0.0,0.0,410.000000,31.000000,1.100000,136.958333,5.550000,12.400000,44.000000,6900.000000,5.000000,0.0,1.0,0.0,0.0,1.0,0.0,1
8,52.000000,100.000000,1.015000,3.000000,0.000000,0.0,1.0,1.0,0.0,138.000000,60.000000,1.900000,133.291667,6.508333,10.800000,33.000000,9600.000000,4.000000,1.0,1.0,0.0,0.0,0.0,1.0,1
9,53.000000,90.000000,1.020000,2.000000,0.000000,1.0,1.0,1.0,0.0,70.000000,107.000000,7.200000,114.000000,3.700000,9.500000,29.000000,12100.000000,3.700000,1.0,1.0,0.0,1.0,0.0,1.0,1


In [38]:
rows_with_nan = df[df.isnull().any(axis=1)]
rows_with_nan

,age,blood_pressure,specific_gravity,albumin,sugar,abnormal_red_blood_cells,abnormal_pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,blood_urea,serum_creatinine,sodium,potassium,haemoglobin,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,poor_appetite,peda_edema,aanemia,class
id,,,,,,,,,,,,,,,,,,,,,,,,,
0,48.0,80.0,1.020,1.0,0.0,NaN,0.0,0.0,0.0,121.0,36.0,1.20,NaN,NaN,15.4,44.0,7800.0,5.2,1.0,1.0,0.0,0.0,0.0,0.0,1
1,7.0,50.0,1.020,4.0,0.0,NaN,0.0,0.0,0.0,NaN,18.0,0.80,NaN,NaN,11.3,38.0,6000.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1
2,62.0,80.0,1.010,2.0,3.0,0.0,0.0,0.0,0.0,423.0,53.0,1.80,NaN,NaN,9.6,31.0,7500.0,NaN,0.0,1.0,0.0,1.0,0.0,1.0,1
4,51.0,80.0,1.010,2.0,0.0,0.0,0.0,0.0,0.0,106.0,26.0,1.40,NaN,NaN,11.6,35.0,7300.0,4.6,0.0,0.0,0.0,0.0,0.0,0.0,1
5,60.0,90.0,1.015,3.0,0.0,NaN,NaN,0.0,0.0,74.0,25.0,1.10,142.0,3.2,12.2,39.0,7800.0,4.4,1.0,1.0,0.0,0.0,1.0,0.0,1
6,68.0,70.0,1.010,0.0,0.0,NaN,0.0,0.0,0.0,100.0,54.0,24.00,104.0,4.0,12.4,36.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1
7,24.0,NaN,1.015,2.0,4.0,0.0,1.0,0.0,0.0,410.0,31.0,1.10,NaN,NaN,12.4,44.0,6900.0,5.0,0.0,1.0,0.0,0.0,1.0,0.0,1
8,52.0,100.0,1.015,3.0,0.0,0.0,1.0,1.0,0.0,138.0,60.0,1.90,NaN,NaN,10.8,33.0,9600.0,4.0,1.0,1.0,0.0,0.0,0.0,1.0,1
10,50.0,60.0,1.010,2.0,4.0,NaN,1.0,1.0,0.0,490.0,55.0,4.00,NaN,NaN,9.4,28.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,1
